### Scalar Functions(Funciones Escalares) -- Um registro para cada linha

In [0]:
use movie_silver

In [0]:
SELECT *, CONCAT(title, ' - ', release_date) AS title_with_release
FROM movies
LIMIT 30;

In [0]:
SELECT SPLIT(name, ' '), SPLIT(name, ' ')[0] NAME, SPLIT(name, ' ')[2] SURNAME
FROM persons
LIMIT 10

In [0]:
SELECT 
    release_date, 
    DATE_FORMAT(release_date, 'dd-MM-yyyy') AS release_date,
    DATE_ADD(release_date, 1) AS `release_date+1`,
    CURRENT_TIMESTAMP()
FROM movies
LIMIT 30;

### Agregate Functions(Funciones de Agregación) -- Um ou Vários registro para cada linha

In [0]:
SELECT 
    year_release_date,
    COUNT(1) AS total_movies,
    MAX(release_date) AS latest_movie,
    SUM(budget) AS total_budget,
    MIN(budget) AS min_budget,
    AVG(budget) AS avg_budget,
    STDDEV(budget) AS stddev_budget,
    COUNT(DISTINCT title) AS total_distinct_movies
FROM movies
GROUP BY year_release_date
HAVING total_movies > 220
ORDER BY year_release_date DESC
LIMIT 30;

### Window Functions(Funciones de Ventana)

In [0]:
SELECT 
    title, 
    year_release_date, 
    release_date, 
    RANK() OVER (PARTITION BY year_release_date ORDER BY release_date) AS RANK
FROM movies
WHERE year_release_date IS NOT NULL
ORDER BY release_date;

### Join entre tablas

In [0]:
CREATE OR REPLACE TEMP VIEW v_movies_2011 
AS
SELECT movie_id, title, budget, revenue, year_release_date, release_date
FROM movies
WHERE year_release_date = 2011

In [0]:
CREATE OR REPLACE TEMP VIEW v_production_countries
AS
SELECT * FROM productions_countries;

In [0]:
CREATE OR REPLACE TEMP VIEW v_countries
AS
SELECT country_id, country_name FROM countries;


### INNER

In [0]:
SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date, VC.country_name
FROM v_movies_2011 VM
INNER JOIN v_production_countries VPC ON VM.movie_id = VPC.movie_id
INNER JOIN v_countries VC ON VPC.country_id = VC.country_id

### LEFT

In [0]:
SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date, VC.country_name
FROM v_movies_2011 VM
LEFT JOIN v_production_countries VPC ON VM.movie_id = VPC.movie_id
LEFT JOIN v_countries VC ON VPC.country_id = VC.country_id
WHERE VC.country_name IS NULL

### RIGHT

In [0]:
SELECT DISTINCT country_name FROM (
SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date, VC.country_name
FROM v_movies_2011 VM
RIGHT JOIN v_production_countries VPC ON VM.movie_id = VPC.movie_id
RIGHT JOIN v_countries VC ON VPC.country_id = VC.country_id
WHERE VM.title IS NULL
)

### FULL JOIN

In [0]:
SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date, VC.country_name
FROM v_movies_2011 VM
FULL JOIN v_production_countries VPC ON VM.movie_id = VPC.movie_id
FULL JOIN v_countries VC ON VPC.country_id = VC.country_id
WHERE VM.title IS NULL OR VC.country_name IS NULL

### SEMI JOIN - Retorna somente os registro do lado esquerdo que tem correspondente no direito
### Idem ao EXISTS

In [0]:
SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date --, VC.country_name
FROM v_movies_2011 VM
INNER JOIN v_production_countries VPC ON VM.movie_id = VPC.movie_id
SEMI JOIN v_countries VC ON VPC.country_id = VC.country_id

##### ANTI JOIN - Retorna somente os registro do lado esquerdo que tem não tem correspondente no direito
##### Idem ao NOT EXISTS

In [0]:
SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date, VPC.movie_id, VPC.country_id
FROM v_movies_2011 VM
LEFT JOIN v_production_countries VPC ON VM.movie_id = VPC.movie_id
ANTI JOIN v_countries VC ON VPC.country_id = VC.country_id

##### CROSS JOIN - Retorna um cartesiano entre o lado esquerdo e o direito

In [0]:
SELECT today, movies.*
FROM (SELECT CURRENT_DATE AS today) AS today_table 
CROSS JOIN 
(SELECT VM.title, VM.budget, VM.revenue, VM.year_release_date, VM.release_date, VM.movie_id
FROM v_movies_2011 VM) As movies